In [1]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy import stats

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [2]:
 # Conexión a BD
# ==============================================================================
import pyodbc

In [3]:
# Configuración de BD y conexión
# ==============================================================================
direccion_servidor = '192.168.1.32,55078'
nombre_bd = 'BDVirunet'
nombre_usuario = 'sa'
password = '123456'

try:
    conexion = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + 
                              direccion_servidor+';DATABASE='+nombre_bd+';UID='+nombre_usuario+';PWD=' + password)
    print("Conexión exitosa")
    
except Exception as e:
    print("Ocurrió un error al conectar a SQL Server: ", e)

Conexión exitosa


In [4]:
dFechaInicio = '2019-01-01'
dFechaFin = '2019-12-31'

In [5]:
try:
    with conexion.cursor() as cursor:
            consulta = "exec STP_OBTENER_TOTALIZADO_PLAGAS ?,?,?,?"
            cursor.execute(consulta, (dFechaInicio, dFechaFin, 1,0))

            rows = cursor.fetchall()
            
            cols = []
            
            for i,_ in enumerate(cursor.description):
                cols.append(cursor.description[i][0])

            pd.DataFrame(np.array(rows), columns = cols)
            
            df = pd.DataFrame(columns =['Evaluador','Fecha','Fundo','Filtrado','Turno','Lote','Grupo_Plaga','Plaga','Caracteristica','Suma','Grado','Promedio','Porcentaje'] )
            for i in range(len(rows)):
                df.loc[i] = list(rows[i])
except Exception as e: 
    print(e)

In [7]:
df["Plaga"].unique()

array(['OLYGONICHUS PUNICAE, OLIGONYCHUS YOTHERSI', 'OIKETICUS KIRBYI',
       'SABULODES AEGROTATA', 'MINADOR', 'TRIPS TABACI', 'BEMISIA TABACI',
       'CONTROLADORES NATURALES', 'PRODIPLOSIS LONGIFILIA',
       'ALEURODICUS COCOIS, ALEURODICUS JULEIKAE',
       'ARGYROTAENIA SPHALEROPA', 'LEPIDOPTEROS (OTROS)',
       'ALEUROPLEUROCELUS', 'OXYDIA VESULIA', 'PULGON',
       'DAGBERTUS MINENSIS, DAGBERTUS PERUANUS'], dtype=object)

In [8]:
dfPlaga =  df[df["Plaga"] == "OLYGONICHUS PUNICAE, OLIGONYCHUS YOTHERSI"]

In [9]:
data_weather = pd.ExcelFile('D:/Proyectos GitHub/Python/Datos/Datos Climaticos Viru Palto.xlsx')
df_weather = data_weather.parse('TABLA')  
data_weather.close()

In [10]:
df_weather = df_weather[df_weather['Año'] == 2019]
df_weather = df_weather[(df_weather['Fecha'] >= dFechaInicio) & (df_weather['Fecha'] <= dFechaFin)]
df_weather.reset_index(drop = True, inplace = True)

In [11]:
df_weather = df_weather[['Fecha','ET(mm).','Tº MIN/DIA (°C)','Tº MAX/DIA (°C)','Rad. Solar Prom.','HUMEDAD PROM.', 'T° Prom/Día (°C)','Semana']]
df_weather = df_weather.rename(columns = {'ET(mm).' : 'ET', 
                                      'Tº MIN/DIA (°C)' : 'T_MIN',
                                     'Tº MAX/DIA (°C)': 'T_MAX',
                                     'Rad. Solar Prom.' : 'RS_PROM',
                                     'HUMEDAD PROM.': 'HM_PROM',
                                      'T° Prom/Día (°C)': 'TMP_PROM',
                                         'Semana': 'Semana'})

In [13]:
dfFilterPlaga = dfPlaga[["Filtrado","Lote","Fecha","Porcentaje"]]

In [14]:
dfPlagaAll = pd.DataFrame(columns = ['FechaClima','ET','T_MIN','T_MAX','RS_PROM','HM_PROM','TMP_PROM','Semana','Filtrado','Lote','Fecha','Porcentaje'])

In [17]:
for filtrado in dfFilterPlaga["Filtrado"].unique():
    dfFiltrado = dfFilterPlaga[dfFilterPlaga["Filtrado"] == filtrado]
    arrLote = dfFiltrado["Lote"].unique()
    for lote in arrLote:
        
        dfPlagaFiltrado = dfFiltrado[dfFiltrado["Lote"] == lote].sort_values(by=['Fecha'])
        dfPlagaFiltrado.reset_index(drop = True, inplace = True)        
        fl_df_weather = df_weather[df_weather['Fecha'].isin(dfPlagaFiltrado['Fecha'])]
        fl_df_weather = fl_df_weather.sort_values(by=['Fecha'])
        fl_df_weather.reset_index(drop = True, inplace = True)
        fl_df_weather = fl_df_weather.rename(columns = {'Fecha' : 'FechaClima'})
        
        df_union = fl_df_weather.join(dfPlagaFiltrado)  
        df_union['Semana'] = pd.to_datetime(df_union['Fecha'], format='%Y-%m-%d').dt.weekofyear
        dfPlagaAll = pd.concat([dfPlagaAll, df_union])

In [19]:
dfPlagaAll.sort_values(by=['Filtrado','Lote','Semana'])

,FechaClima,ET,T_MIN,T_MAX,RS_PROM,HM_PROM,TMP_PROM,Semana,Filtrado,Lote,Fecha,Porcentaje
0,2019-01-03,3.200,20.4,27.4,187.1,86.0,22.870000,1,10N,1006,2019-01-03,2
1,2019-01-30,4.300,22.1,30.8,228.9,83.0,24.822917,5,10N,1006,2019-01-30,71
2,2019-02-07,4.400,22.2,29.8,259.2,84.0,25.462500,6,10N,1006,2019-02-07,71
3,2019-05-13,1.800,18.8,25.1,106.3,85.0,21.683333,20,10N,1006,2019-05-13,68
4,2019-05-20,3.300,16.9,25.5,207.5,84.0,20.658333,21,10N,1006,2019-05-20,94
...,...,...,...,...,...,...,...,...,...,...,...,...
11,2019-11-25,1.118,18.2,18.2,69.0,88.0,19.900000,48,9N,995,2019-11-25,210
12,2019-12-05,3.900,17.8,27.1,236.3,87.0,21.593750,49,9N,995,2019-12-05,174
13,2019-12-09,3.600,18.3,25.5,223.5,85.0,21.345833,50,9N,995,2019-12-09,264
14,2019-12-20,3.100,19.0,25.1,192.5,84.0,21.218750,51,9N,995,2019-12-20,240


In [20]:
dfPlagaAll = dfPlagaAll[["Filtrado","Lote","Semana","Fecha","Porcentaje","ET","T_MIN","T_MAX","RS_PROM","HM_PROM","TMP_PROM"]]

In [21]:
dfPlagaAll.to_excel("datasets/Arañita2019.xlsx")